<h1>Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Example-program" data-toc-modified-id="Example-program-1">Example program</a></span></li><li><span><a href="#Built-in-tests" data-toc-modified-id="Built-in-tests-2">Built-in tests</a></span></li><li><span><a href="#Test-functions" data-toc-modified-id="Test-functions-3">Test functions</a></span><ul class="toc-item"><li><span><a href="#Assertions" data-toc-modified-id="Assertions-3.1">Assertions</a></span></li></ul></li><li><span><a href="#Test-runners" data-toc-modified-id="Test-runners-4">Test runners</a></span></li></ul></div>

In [1]:
import os

os.chdir('examples')

# Testing

As you have probably noticed by now, it is easy to make mistakes when writing a computer program. Computers cannot easily guess what it is that we want of them, nor do they have any idea of what sort of things it is reasonable to want to do. So we must give a computer precise, [syntactically correct](extras/glossary.md#syntax) instructions that represent exactly what we want to do, and if we get this wrong, the computer will either be unable to do as we asked, in which case our program will break off with an [error](extras/glossary.md#error), or it will happily carry out the instructions we gave it, even when these are not actually the instructions that we wanted to give, and even if we have mistakenly instructed it to do something stupid or malicious such as emailing the contents of our downloads folder to our boss. It is therefore important to check our programs carefully for mistakes.

Up until now, we have been able to check a program fairly easily by simply running it, if it is just a [script](extras/glossary.md#script), or importing it and then using its functions in the console if it is a [module](extras/glossary.md#module). But this sort of informal testing can only take us so far. Once we start writing programs that may take multiple different actions depending on user input, subtle mistakes may be difficult to detect in a quick informal test, because they only arise for some inputs and not others. For these reasons, most software developers test their programs systematically as they go along. Instead of testing a program manually, they write a second program whose only purpose is to test the main program. This test program tries out the main program with different inputs, and verifies that its behavior is as expected. In other words, a test program automates the process of testing the main program.

Writing test programs can be tedious. As if we didn't already have enough work to do writing the main program, we now have to write another one that won't even be part of the finished product. It is tempting to dispense with testing, especially if a project is small. But we should not. Good tests act as a reasonable guarantee that our program functions correctly. And as the saying goes: Most customers prefer a product that actually works.

## Example program

In the spirit of the class so far, we won't actually be developing a product that any healthy customer would want to download. Instead, we have a program for producing [spoonerisms](https://en.wikipedia.org/wiki/Spoonerism). A spoonerism is a play on words in which the initial consonants (or groups of consonants) of two words are swapped. For example:

* crushing blow → blushing crow
* cosy nook → nosy cook
* wasted term → tasted werm

Since this lesson is about testing, we will look at the finished example program already. Our task will be to write some tests for the program. Let's import the program as [module](extras/glossary.md#module) and make some informal tests first. (If you need to refresh your understanding of modules and imports, take a look back at the [lesson on modules](modules.ipynb).)

In [2]:
import spoonerisms

dir(spoonerisms)

['VOWELS',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 'find_first_vowel',
 'spoonerize']

The main function is `spoonerize()`. We can look at its [docstring](extras/glossary.md#docstring) using the `help()` function:

In [3]:
help(spoonerisms.spoonerize)

Help on function spoonerize in module spoonerisms:

spoonerize(wordpair)
    Spoonerize a word pair.
    
    A spoonerism switches the initial consonant clusters of words.
    
    Example:
        >>> spoonerize('smart fella')
        'fart smella'
    
    Argument:
        wordpair: A string containing exactly two words.
    
    Returns:
        A string containing the spoonerized phrase.
    
    Raises ValueError:
        If wordpair does not contain exactly two words.



Let's test the example input given in the docstring:

In [4]:
spoonerisms.spoonerize('smart fella')

'fart smella'

This module is very slightly more complex than those we have encountered so far. In addition to the main function, it includes an additional 'helper function' that is [called](extras/glossary.md#call) from within the main function, and whose purpose is to find the location of the first vowel in a word (according to Python's zero-based [indexing](extras/glossary.md#index)). For example:

In [5]:
spoonerisms.find_first_vowel('smart')

2

This helper function in turn depends on a variable called `VOWELS`, a string containing the characters considered to be vowels. These could just have been defined within the `find_first_vowel()` function, but including them as a variable allows us later to add more vowels easily if we want to extend the module's capabilities.

This is a good point to start writing tests; our program has gone beyond just a single function, and we might want to add to its capabilities in the future. Before we begin writing the tests, you might want to take a look at the module file [spoonerisms.py](examples/spoonerisms.py) and quickly assure yourself that you understand how it works.

## Built-in tests

The module in fact already incorporates one short test of its workings. It uses the `__name__` special variable to print out a result when the module is run as the main program. This is something that we learned about in the [lesson on modules](modules.ipynb#__name__). This sort of 'built-in test' can take us quite far, and is especially helpful early on in the development of a program. We will learn about a few ways in which a separate test program can be more convenient as development goes on. But first we need to cover the basics of test programs.

## Test functions

A test program goes in a separate file. This file usually has the same name as the file that it tests, but prefixed with *test_*. So in our case our test file will be called *test_spoonerisms.py*. Among the first things that the test program should do is import the module to be tested, since it will need to [call](extras/glossary.md#call) that module's functions and check their results. After importing the module to be tested, the test program defines functions, each of which tests one aspect of the module.

Test functions look a little different from the functions we have written so far. They do not have any [return value](extras/glossary.md#return), and in most cases they have no input [arguments](extras/glossary.md#argument) either. In place of a return value, most test functions instead make an [assertion](extras/glossary.md#assertion).

### Assertions

What is an assertion? An assertion is yet another kind of [control statement](extras/glossary.md#control). You can think of it as a special kind of [condition](extras/glossary.md#condition), like in an `if` statement. Similar to an `if` statement, an assertion checks whether a particular condition is true or false, for example checking whether two things are equal using `==`. But unlike an `if` statement, we do not specify what happens when the condition is true and what happens when it is false. Instead, an assertion has fixed consequences: If the condition is true, nothing happens and the program continues as normal; if the condition is not true, an [exception](extras/glossary.md#exception) is raised.

An assertion begins with the [keyword](extras/glossary.md#keyword) `assert`. Here is an example of an assertion that is true:

In [6]:
assert 2 + 2 == 4

As you can see, nothing happens when Python runs this line. A true assertion just 'passes' and the Python interpreter moves on to the next line of the program.

Compare this with what happens when an assertion is untrue. We get an `AssertionError`:

In [7]:
assert 2 + 2 == 5

AssertionError: 

We can therefore use an assertion to check whether the [return value](extras/glossary.md#return) of a function is as expected. For example for the `spoonerize()` function from our example module:

In [8]:
result = spoonerisms.spoonerize('smart fella')

assert result == 'fart smella'

The simplest test functions are just functions that contain one assertion. No input arguments or return value are needed; the function's only purpose is to alert us by raising an exception if things are not working as expected. Like test files, the names of test functions should be prefixed with `test_`, and then state what is being tested. Here is an example for our `spoonerize()` function, turning the assertion above into a test function.

(If you need first to remind yourself of the syntax for functions, take a look back at the [lesson on functions](functions.ipynb#Defining-functions).)

In [9]:
def test_spoonerize():
    result = spoonerisms.spoonerize('smart fella')
    assert result == 'fart smella'

When we [call](extras/glossary.md#call) our test function, no exception is raised, confirming that the assertion was true.

In [10]:
test_spoonerize()

Take a look at the finished test file for our example module, [test_spoonerisms.py](examples/test_spoonerisms.py). You will see that it first imports the *spoonerisms* module, then defines several test functions like the one above. Each of these tests one aspect of the workings of the *spoonerisms* module. (There is one slightly different test function at the very end of the file. Ignore this for now; we will come to it in a moment.)

If we would like to test any aspect of the *spoonerisms* module, we can import the test module and run its functions, confirming that no exceptions occur:

In [11]:
import test_spoonerisms

test_spoonerisms.test_find_first_vowel()
test_spoonerisms.test_spoonerize()

## Test runners

Our solution so far isn't entirely satisfactory. We wanted to have an *automated* test of our module, so that we can for example quickly run the tests each time we make changes, and make sure that our changes haven't broken anything. Importing the test module and then running each of its tests individually is very laborious. Because automated testing is such a common task in programming, most programming languages provide tools for running multiple tests with a single command. These tools are sometimes termed 'test runners'.

### pytest

Python's built-in testing tools are provided in a module called `unittest` in the [standard library](standard_library.ipynb). However, there are also several excellent [third-party packages](standard_library.ipynb#Third-party-packages) that provide testing tools.

In [13]:
! pytest test_spoonerisms.py

============================= test session starts ==============================
platform linux -- Python 3.6.9, pytest-5.3.2, py-1.8.1, pluggy-0.13.1
rootdir: /home/lt/GitHub/introduction-to-programming/content/examples
collected 5 items                                                              

test_spoonerisms.py .....                                                [100%]

============================== 5 passed in 0.01s ===============================
